In [1]:
#!/usr/bin/env python
# coding: utf-8

import os
import serial
import threading
import datetime
import gspread
from google.oauth2.service_account import Credentials
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import cv2

# Google Sheets setup
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
COLUMN_HEADERS = [
    "Timestamp", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Global stop event for the monitoring thread
stop_event = threading.Event()

class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("FED3 Monitor")
        self.root.geometry("800x480")  # Adjust for a 7-inch touchscreen

        # GUI Variables
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.json_path = tk.StringVar()
        self.spreadsheet_id = tk.StringVar()
        self.save_path = ""
        self.camera = None
        self.serial_port = None
        self.gspread_client = None
        self.sheet = None

        self.setup_gui()

    def setup_gui(self):
        # GUI Layout
        tk.Label(self.root, text="Experimenter Name:", font=("Cascadia Code", 8)).grid(column=0, row=0, sticky=tk.E)
        tk.Entry(self.root, textvariable=self.experimenter_name, width=20).grid(column=1, row=0, sticky=tk.W)

        tk.Label(self.root, text="Experiment Name:", font=("Cascadia Code", 8)).grid(column=2, row=0, sticky=tk.E)
        tk.Entry(self.root, textvariable=self.experiment_name, width=20).grid(column=3, row=0, sticky=tk.W)

        tk.Label(self.root, text="Google API JSON File:", font=("Cascadia Code", 8)).grid(column=0, row=1, sticky=tk.E)
        tk.Entry(self.root, textvariable=self.json_path, width=30).grid(column=1, row=1, columnspan=2, sticky=tk.W)
        tk.Button(self.root, text="Browse", command=self.browse_json).grid(column=3, row=1, sticky=tk.W)

        tk.Label(self.root, text="Google Spreadsheet ID:", font=("Cascadia Code", 8)).grid(column=0, row=2, sticky=tk.E)
        tk.Entry(self.root, textvariable=self.spreadsheet_id, width=30).grid(column=1, row=2, columnspan=2, sticky=tk.W)

        tk.Button(self.root, text="Select Save Folder", command=self.browse_folder, bg="gold").grid(column=3, row=3)

        tk.Button(self.root, text="START", font=("Cascadia Code", 10, "bold"), bg="green", fg="white", command=self.start_monitoring).grid(column=0, row=4, padx=5, pady=5, sticky=tk.W)
        tk.Button(self.root, text="STOP", font=("Cascadia Code", 10, "bold"), bg="red", fg="white", command=self.stop_monitoring).grid(column=1, row=4, padx=5, pady=5, sticky=tk.W)

    def browse_json(self):
        self.json_path.set(filedialog.askopenfilename(title="Select JSON File"))

    def browse_folder(self):
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Videos")

    def start_monitoring(self):
        # Ensure save path is selected
        if not self.save_path:
            messagebox.showerror("Error", "Please select a folder to save data.")
            return

        # Initialize Google Sheets
        try:
            creds = Credentials.from_service_account_file(self.json_path.get(), scopes=SCOPE)
            self.gspread_client = gspread.authorize(creds)
            spreadsheet = self.gspread_client.open_by_key(self.spreadsheet_id.get())
            self.sheet = spreadsheet.sheet1  # Use the first sheet
            self.sheet.append_row(COLUMN_HEADERS)
            print("Connected to Google Sheets")
        except Exception as e:
            messagebox.showerror("Error", f"Error setting up Google Sheets API: {e}")
            return

        # Initialize Camera
        self.camera = cv2.VideoCapture(0)  # Open /dev/video0
        if not self.camera.isOpened():
            messagebox.showerror("Error", "Failed to open the camera.")
            return

        # Initialize Serial Port
        try:
            self.serial_port = serial.Serial('/dev/ttyACM0', 115200, timeout=1)
            print("Connected to serial port /dev/ttyACM0")
        except Exception as e:
            messagebox.showerror("Error", f"Error connecting to serial port: {e}")
            return

        # Start monitoring in a separate thread
        self.monitor_thread = threading.Thread(target=self.monitor_fed)
        self.monitor_thread.start()

    def monitor_fed(self):
        while not stop_event.is_set():
            try:
                # Read data from serial port
                line = self.serial_port.readline().decode('utf-8').strip()
                if "pellet" in line.lower():
                    print("Pellet event detected.")
                    self.record_video()  # Trigger video recording for 20 seconds
                    self.log_to_google_sheets(line)
            except Exception as e:
                print(f"Error reading serial data: {e}")

    def log_to_google_sheets(self, data):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
        data_list = data.split(",")
        row = [timestamp] + data_list
        if len(row) == len(COLUMN_HEADERS):
            try:
                self.sheet.append_row(row)
                print(f"Data logged to Google Sheets: {row}")
            except Exception as e:
                print(f"Error logging data to Google Sheets: {e}")

    def record_video(self):
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = os.path.join(self.save_path, f"camera_{timestamp}.avi")

        # Set up video writer for 20-second recording
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(filename, fourcc, 20.0, (640, 480))

        # Capture 20 seconds of video
        for _ in range(400):  # 20 seconds at 20 FPS
            ret, frame = self.camera.read()
            if ret:
                out.write(frame)
            else:
                print("Error reading from camera.")
                break

        out.release()
        print(f"Video saved as {filename}")

    def stop_monitoring(self):
        stop_event.set()
        if self.monitor_thread.is_alive():
            self.monitor_thread.join()
        if self.camera:
            self.camera.release()
        if self.serial_port:
            self.serial_port.close()
        print("Monitoring stopped.")

# Main execution
if __name__ == "__main__":
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()


Connected to Google Sheets
Connected to serial port /dev/ttyACM0
Pellet event detected.
Video saved as /home/hpfed/Desktop/DATA_MAIN/camera_20241114_160251.avi
Pellet event detected.
Video saved as /home/hpfed/Desktop/DATA_MAIN/camera_20241114_160312.avi
Monitoring stopped.
